In [1]:
!pip install transformers torch safetensors


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_directory = r'C:\Users\Het\Desktop\Het\Text_classification\trained_model'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_directory)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_directory, use_safetensors=True)

In [18]:
extracted_text = """
 it is eight years despite filing affidavits in since hc issued a bombay high court assuring series of directions that they will not erect illegal ordering the bmc and hoardings political parties the government to take continue to deface the action against metropolis in contempt of the hoardings judiciary eknath recently on both authorities shindes birthday the city was appear to be swathed in hoardings greeting taking these ravikiran deshmukh the deputy chief minister instructions lightly remili
"""

In [19]:
inputs = tokenizer(extracted_text, truncation=True, padding=True, return_tensors="pt")

In [20]:
import torch

# Ensure the model is in evaluation mode
model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()

# Map the predicted class ID to the corresponding label
labels = ['politics', 'sports', 'business', 'entertainment']  # Adjust based on your model's training
predicted_label = labels[predicted_class_id]

print(f"Predicted Category: {predicted_label}")

Predicted Category: politics
